In [ ]:
from depmapomics.tracker import initTracker
from gsheets import Sheets

In [ ]:
frame = Sheets.from_files( "~/.client_secret.json", "~/.storage.json").get("https://docs.google.com/spreadsheets/d/1esHqP0Z76_oVMhQtaigyF9FuuniAQefkANoUzLD9rt4").sheets[1].to_frame(index_col=0, skiprows=2)

In [ ]:
res = frame[['VisualCharacteristics','GrowthPattern']]

In [ ]:
res

In [ ]:
def isin(li, st):
    for val in li:
        if val in st:
            return True
    return False

In [ ]:
val is np.nan

In [ ]:
val

In [ ]:
res.VisualCharacteristics.iloc[:10]

In [ ]:
dat = {i: [] for i in ["forma", "group", "shape", "debris", "size", "characteristics", "other"]}
for val in res.VisualCharacteristics.str.lower():
    if val is None or val is np.nan:
        for i in ["forma", "group", "shape", "debris", "size", "characteristics", "other"]:
            dat[i].append('')
        continue
    if isin(["loosely adherent", "semi-adherent", "accasional adherent", "mixed", "mix", "occasional adherent", "monolayer and suspension", "monolayer and floating"], val):
        dat["forma"].append("mixed")
    elif isin(["adherent", "adherenet", "monolayer"], val):
        if "monolayer" in val:
            dat["forma"].append("monolayer")
        else:
            dat['forma'].append('adherent')
    elif isin(["suspension", "suspention"], val):
        dat["forma"].append("suspension")
    else:
        dat["forma"].append("")
    
    if isin(["islets", "aggregates", "clumps"], val):
        dat["group"].append('aggregates')
    elif isin(["single", "singly"], val):
        dat["group"].append('single')
    else:
        dat["group"].append('')
    
    if isin(["round cells", "spheroidal", "spheroid", "round"], val):
        dat["shape"].append("round")
    elif isin(["polymorph", "polymorphic", "polypoid", "variable morphology", "pleomorphic"], val):
        dat["shape"].append("polymorph")
    elif "polygonal" in val:
        dat["shape"].append("polygonal")
    elif "spindle" in val:
        dat["shape"].append("spindle")
    else:
        dat["shape"].append("")
    
    if isin(["debris", "dark", "granules"], val):
        dat["debris"].append('true')    
    else:
        dat["debris"].append('')   
    
    if isin(["gigantic","giant"], val):
        dat["size"].append('gigantic')
    elif "large" in val:
        dat["size"].append('large')
    elif "small" in val:
        dat["size"].append("small")
    else:
        dat['size'].append('')
    
    if "neuroblast" in val:
        dat["characteristics"].append("neuroblast")
    elif isin(["epithelial", "epitheloid"], val):
        dat["characteristics"].append("epithelial")
    elif isin(["lymphoblast", "lymphoblastoid", "lymphocyte"], val):
        dat["characteristics"].append("lymphocyte")
    elif isin(["fibroblst", "fibroblast", "sarcomatoid"], val):
        dat["characteristics"].append("fibroblast")
    elif "keratinocyte" in val:
        dat["characteristics"].append("keratinocyte")
    elif "plamacyte" in val:
        dat["characteristics"].append("plamacyte")
    else:
        dat["characteristics"].append("")
    
    if "long extentions" in val:
        dat["other"].append('with_extensions')
    else:
        dat["other"].append('')

In [ ]:
set(res.GrowthPattern)

In [ ]:
for i, val in enumerate(res.GrowthPattern.str.lower()):
    if val=="3d":
        dat['forma']=='3d'
    elif val in ["adherent", "loose_adherent"]:
        if dat['forma'][i]=="suspension":
            dat['forma'][i]=""
        elif dat['forma'][i]=="":
            dat['forma'][i]="adherent"
    elif val=="mix":
        dat['forma'][i]="mixed"
    elif val in ["suspension", "suspension_agg"]:
        if dat['forma'][i]=="adherent":
            dat['forma'][i]=""
        elif dat['forma'][i]=="":
            dat['forma'][i]="suspension"
        
        if val=="suspension_agg":
            if dat["group"][i]=="single":
                dat["group"][i]=""
            elif dat["group"][i]=="":
                dat["group"][i]="aggregate"

In [ ]:
pd.DataFrame(dat).to_csv(sys.stdout, sep='\t', index=None)

In [ ]:
set(res.VisualCharacteristics.str.lower().str.replace(',', ' ').str.replace( ';', ' ').str.replace('  ', ' '))